In [8]:
import requests
import pandas as pd

In [15]:
LAST_FM_KEY = 'your lastfm key'
USERNAME_LAST = 'your lastfm username'
key = LAST_FM_KEY
username = USERNAME_LAST
client_id = 'your spotify client id'
client_secret = 'your spotify client secret'

In [16]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [17]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                      client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [18]:
def get_scrobbles(username=username, api_key=key, limit=200, page=1):
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={}&api_key={}&limit={}&page={}&format=json'

    responses = []
    artist_names = []
    track_names = []

    request_url = url.format(username, api_key, limit, page)
    response = requests.get(request_url).json()
    pages = int(response['recenttracks']['@attr']['totalPages'])

    print('{} total pages to retrieve'.format(pages))

    for page in range(1, int(pages) + 1):
        if page % 10 == 0:
            print(page)
        request_url = url.format(username, api_key, limit, page)
        responses.append(requests.get(request_url))

    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles['recenttracks']['track']:
            artist_names.append(scrobble['artist']['#text'])
            track_names.append(scrobble['name'])
    
    return artist_names, track_names

In [19]:
def get_spotify_uris():
    saved_uris = []
    artist_names, track_names = get_scrobbles()
    
    for i in range(len(artist_names)):
        artist = artist_names[i]
        track = track_names[i]
        q = 'artist:{} track: {}'.format(artist, track)
        results = sp.search(q=q, limit=1, type='track')
        uri = results['tracks']['items'][0]['uri']
        saved_uris.append(uri)
        
    return saved_uris

In [20]:
def get_audio_features():
    uris = get_spotify_uris()
    
    artist = []
    track = []
    danceability = []
    energy = []
    key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    urri = []
    duration_ms = []

    for uri in uris:
        x = sp.audio_features(uri)
        y = sp.track(uri)
        for audio_features in x:
            danceability.append(audio_features['danceability'])
            energy.append(audio_features['energy'])
            key.append(audio_features['key'])
            loudness.append(audio_features['loudness'])
            mode.append(audio_features['mode'])
            speechiness.append(audio_features['speechiness'])
            acousticness.append(audio_features['acousticness'])
            instrumentalness.append(audio_features['instrumentalness'])
            liveness.append(audio_features['liveness'])
            valence.append(audio_features['valence'])
            tempo.append(audio_features['tempo'])
            urri.append(audio_features['uri'])
            duration_ms.append(audio_features['duration_ms'])
            print('append')
      
        artist.append(y['album']['artists'][0]['name'])
        track.append(y['name'])

    df = pd.DataFrame()
    df['artist'] = artist
    df['track'] = track
    df['danceability'] = danceability
    df['energy'] = energy
    df['key'] = key
    df['loudness'] = loudness
    df['mode'] = mode
    df['speechiness'] = speechiness
    df['acousticness'] = acousticness
    df['instrumentalness'] = instrumentalness
    df['liveness'] = liveness
    df['valence'] = valence
    df['tempo'] = tempo
    df['urri'] = urri
    df['duration_ms'] = duration_ms
    
    df.to_csv('enter directory path')
    
    return df